# Example - Multi-lingual semantic search

# Lancedb Embeddings API: Multi-lingual semantic search
In this example, we'll build a simple LanceDB table containing embeddings for different languages that can be used for universal semantic search.
* The **Dataset** used will be wikipedia dataset in English and French
* The **Model** used will be cohere's multi-lingual model

In this example, we'll explore LanceDB's Embeddings API that allows you to create tables that automatically vectorize data once you define the config at the time of table creation. Let's dive right in!

To learn more about LanceDB, visit [our docs](https://lancedb.github.io/lancedb/)


In [1]:
!pip install -qU datasets cohere openai lancedb



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


## Create datasets
For accessing the datasets, we'll use datasets library in streaming mode. We'll use english and french versions and embed them together. For semantic search the order should be irrelevant

In [11]:
from datasets import load_dataset

en = dataset = load_dataset("wikipedia", "20220301.en", streaming=True,)
fr = load_dataset("wikipedia", "20220301.fr", streaming=True)

datasets = {"english": iter(en['train']), "french": iter(fr['train'])}

/Users/ayush/vectordb-recipes/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's take a look at the dataset format

In [38]:
next(iter(en['train']))

{'id': '12',
 'url': 'https://en.wikipedia.org/wiki/Anarchism',
 'title': 'Anarchism',
 'text': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.\n\nHumans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latte

In [39]:
next(iter(fr['train']))

{'id': '3',
 'url': 'https://fr.wikipedia.org/wiki/Antoine%20Meillet',
 'title': 'Antoine Meillet',
 'text': "Paul Jules Antoine Meillet, né le  à Moulins (Allier) et mort le  à Châteaumeillant (Cher), est le principal linguiste français des premières décennies du . Il est aussi philologue.\n\nBiographie \nD'origine bourbonnaise, fils d'un notaire de Châteaumeillant (Cher), Antoine Meillet fait ses études secondaires au lycée de Moulins.\n\nÉtudiant à la faculté des lettres de Paris à partir de 1885 où il suit notamment les cours de Louis Havet, il assiste également à ceux de Michel Bréal au Collège de France et de Ferdinand de Saussure à l'École pratique des hautes études.\n\nEn 1889, il est major de l'agrégation de grammaire.\n\nIl assure à la suite de Saussure le cours de grammaire comparée, qu'il complète à partir de 1894 par une conférence sur les langues persanes.\n\nEn 1897, il soutient sa thèse pour le doctorat ès lettres (Recherches sur l'emploi du génitif-accusatif en vieux-s

## LanceDB Embeddings API
Let's see how you can use the embeddings API to create an ingestion pipeline that automatically does all the vectorization for you both when ingesting new data or searching queries.

### OpenAI API Example
Let us take a look at openAI example first. LanceDB comes with OpenAI embedding function support.
* Create the instance of the available embedding function or create your own
* Create the scheme of the table, marking source end vector fields. Each embedding function can have multiple source and vector feilds
* Create a table with schema

Doing this creates a table with where embedding function information is ingested as metadata so you can forget about all the modelling details and focus only ingesting and retrieving data.

In [ ]:
import os
import lancedb
import getpass
from lancedb.embeddings import EmbeddingFunctionRegistry
from lancedb.pydantic import LanceModel, Vector

if "OPENAI_API_KEY" not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")
    
registry = EmbeddingFunctionRegistry().get_instance()
openai = registry.get("openai").create() # uses multi-lingual model by default (768 dim)

class Schema(LanceModel):
    vector: Vector(openai.ndims()) = openai.VectorField()
    text: str = openai.SourceField()
    url: str
    title: str
    id: str
    lang: str

db = lancedb.connect("~/lancedb")
tbl_openai = db.create_table("wikipedia-openai", schema=Schema, mode="overwrite")

### Cohere Embedding Table
Now let's see another example using cohere embedding function which is also supported directly by LanceDB. We will follow the same steps.

In [29]:
import os
import lancedb
import getpass
from lancedb.embeddings import EmbeddingFunctionRegistry
from lancedb.pydantic import LanceModel, Vector

if "COHERE_API_KEY" not in os.environ:
    os.environ['COHERE_API_KEY'] = getpass.getpass("Enter your Cohere API key: ")
    
registry = EmbeddingFunctionRegistry().get_instance()
cohere = registry.get("cohere").create() # uses multi-lingual model by default (768 dim)

class Schema(LanceModel):
    vector: Vector(cohere.ndims()) = cohere.VectorField()
    text: str = cohere.SourceField()
    url: str
    title: str
    id: str
    lang: str

db = lancedb.connect("~/lancedb")
tbl_cohere = db.create_table("wikipedia-cohere", schema=Schema, mode="overwrite")

## Ingest data
Now, we have the table set up for ingesting the dataset. 

In [30]:
from tqdm.auto import tqdm
import time
# let's use cohere embeddings. Use can also set it to openai version of the table
tbl = tbl_cohere
batch_size = 1000
num_records = 10000
data = []

for i in tqdm(range(0, num_records, batch_size)):

    for lang, dataset in datasets.items():
        
        batch = [next(dataset) for _ in range(batch_size)]
        
        texts = [x['text'] for x in batch]
        ids = [f"{x['id']}-{lang}" for x in batch]
        data.extend({
           'text': x['text'], 'title': x['title'], 'url': x['url'], 'lang': lang, 'id': f"{lang}-{x['id']}"
        } for x in batch)

    # add in batches to avoid token limit
    tbl.add(data)
    data = []
    time.sleep(20) # wait for 20 seconds to avoid rate limit

100%|██████████| 10/10 [06:10<00:00, 37.08s/it]


## Searching multi-lingual embedding space
Let us now search the table with a substring from a random batch in french

In [12]:
it = iter(fr['train'])
for i in range(5):
    next(it)
query = next(it)
query

{'id': '12',
 'url': 'https://fr.wikipedia.org/wiki/Arm%C3%A9e%20r%C3%A9publicaine%20irlandaise',
 'title': 'Armée républicaine irlandaise',
 'text': "L'Armée républicaine irlandaise (, IRA ; ) est le nom porté, depuis le début du , par plusieurs organisations paramilitaires luttant par les armes contre la présence britannique en Irlande du Nord. Les différents groupes se référent à eux comme Óglaigh na hÉireann (« volontaires d'Irlande »).\n\n L' appelée aussi Old IRA, issue de l'union en 1916 entre l' (proche du Parti travailliste irlandais) et les Irish Volunteers (alors généralement proches de l'IRB), est active entre  et , pendant la guerre d'indépendance irlandaise. Si ceux qui ont accepté le traité anglo-irlandais forment les Forces de Défense irlandaises, une partie de l'organisation, refusant cet accord, se constitue en une nouvelle Irish Republican Army, illégale.\n L'Irish Republican Army anti-traité apparaît entre avril et  du fait du refus du traité anglo-irlandais par une

Let's take the first line from the above text body:
```
L'Armée républicaine irlandaise (, IRA ; ) est le nom porté, depuis le début du , par plusieurs organisations paramilitaires luttant par les armes contre la présence britannique en Irlande du Nord.
```
This translates to the following in english
```
The Irish Republican Army (, IRA; ) is the name worn, since the beginning of the 19th century, by several paramilitary organizations fighting with arms against the British presence in Northern Ireland.
```

Let us now see what at the results that are semantically closer to this in our dataset.

In [21]:
import os
import getpass
import lancedb

if "COHERE_API_KEY" not in os.environ:
    os.environ['COHERE_API_KEY'] = getpass.getpass("Enter your Cohere API key: ")


You can now load the table even in a different session and anything ingest or search will be automatically vectorized. Let us now run the query.

In [23]:
%%timeit

db = lancedb.connect("~/lancedb")
tbl = db.open_table("wikipedia-cohere") # We just open the existing
rs = tbl.search(query["text"]).limit(3).to_list()

469 ms ± 39.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
for r in rs:
    print(f" **TEXT id-{r['id']}** \n {r['text']} \n")
#

 **TEXT id-french-12** 
 L'Armée républicaine irlandaise (, IRA ; ) est le nom porté, depuis le début du , par plusieurs organisations paramilitaires luttant par les armes contre la présence britannique en Irlande du Nord. Les différents groupes se référent à eux comme Óglaigh na hÉireann (« volontaires d'Irlande »).

 L' appelée aussi Old IRA, issue de l'union en 1916 entre l' (proche du Parti travailliste irlandais) et les Irish Volunteers (alors généralement proches de l'IRB), est active entre  et , pendant la guerre d'indépendance irlandaise. Si ceux qui ont accepté le traité anglo-irlandais forment les Forces de Défense irlandaises, une partie de l'organisation, refusant cet accord, se constitue en une nouvelle Irish Republican Army, illégale.
 L'Irish Republican Army anti-traité apparaît entre avril et  du fait du refus du traité anglo-irlandais par une partie de l'Old IRA. Elle participe ainsi à la guerre civile irlandaise de  à . Elle maintient son activité dans les deux Irland

As you can see in the above result, the closest match is the text itself that we used to search. The second closest match in an English text with a similar semantic meaning referring to IRA. This is what a multi-lingual embedding model can do.

Find more examples on [VectorDB-recipes](https://github.com/lancedb/vectordb-recipes) repo